In [1]:
import pandas as pd
import numpy as np

In [2]:
#Reading CSV file
df = pd.read_csv('~/Desktop/ds_bootcamp/week3/project/data/_data.csv')
pd.set_option('display.max_columns', None)

In [3]:
#Deleting Unnecessary columns from data frame
df = df.drop(['Unnamed: 0', 'Тип'], axis=1)
df = df.drop(['Парковка'], axis=1) # Убераем парковку
df = df.drop(['Телефоны'], axis=1) # Убераем телефоны
df = df.drop(['Площадь комнат, м2'], axis=1) # Убераем площади комнат

In [4]:
#Renaming columns from Russian to English
df = df.rename(columns={'ID  объявления': 'post_id', 'Количество комнат': 'number_of_rooms', 
                            'Метро': 'metro', 'Адрес': 'address', 'Площадь, м2': 'area_m2', 'Дом': 'apt_short_desc'})

In [5]:
#Getting the number of items in list by region
lst_add = [address.split(', ')[0] for address in df['address']]
series_add = pd.Series(lst_add)
unique_counts = series_add.value_counts()
unique_counts

Москва                19737
Санкт-Петербург        3028
Краснодарский край      603
Name: count, dtype: int64

In [6]:
#Getting indexes of the rows which are not in Moscow
indexes_not_moscow = [i for i, address in enumerate(lst_add) if address != 'Москва']
indexes_not_moscow_list = indexes_not_moscow
indexes_not_moscow_list

[884,
 885,
 886,
 887,
 888,
 889,
 890,
 891,
 892,
 893,
 894,
 895,
 896,
 897,
 898,
 899,
 900,
 901,
 902,
 903,
 904,
 905,
 906,
 907,
 908,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 919,
 920,
 921,
 922,
 923,
 924,
 925,
 926,
 927,
 928,
 929,
 930,
 931,
 932,
 933,
 934,
 935,
 936,
 937,
 938,
 939,
 940,
 941,
 942,
 943,
 944,
 945,
 946,
 947,
 948,
 949,
 950,
 951,
 952,
 953,
 954,
 955,
 956,
 957,
 958,
 959,
 960,
 961,
 962,
 963,
 964,
 965,
 966,
 967,
 968,
 969,
 970,
 971,
 972,
 973,
 974,
 975,
 976,
 977,
 978,
 979,
 980,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990,
 991,
 992,
 993,
 994,
 995,
 996,
 997,
 998,
 999,
 1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1024,
 1025,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1034,
 1035,
 1036,
 1037,
 1038,
 1039,
 1040,
 1041,
 1042,
 10

In [7]:
#Replacing nan values by the most frequent element 
df['number_of_rooms'] = df['number_of_rooms'].replace(to_replace=[None], value=df.loc[:, 'number_of_rooms'].mode()[0])

In [8]:
#Creating 3 new features in data frame
df.insert(2, 'isolated_apt', 0)
df.insert(3, 'adjacent_apt', 0)
df.insert(4, 'iso_adj_apt', 0)

In [9]:
#Filling in 3 new columns by 1 if Apt has specific features related to columns 
splited_list_of_num_rooms = [i.split(', ') for i in df['number_of_rooms']]

for i in range(len(splited_list_of_num_rooms)):
    if splited_list_of_num_rooms[i][0].isdigit():
        df.loc[i, 'number_of_rooms'] = splited_list_of_num_rooms[i][0]
        
        if len(splited_list_of_num_rooms[i]) > 1:
            if splited_list_of_num_rooms[i][1] == 'Изолированная':
                df.loc[i, 'isolated_apt'] = 1
            
            if splited_list_of_num_rooms[i][1] == 'Смежная':
                df.loc[i, 'adjacent_apt'] = 1
            
            if splited_list_of_num_rooms[i][1] == 'Оба варианта':
                df.loc[i, 'iso_adj_apt'] = 1

In [10]:
#Replacing nan values by the most frequent element 
df['metro'] = df['metro'].replace(to_replace=[None], value=df.loc[:, 'metro'].mode()[0])

In [11]:
#Adding new columns related to Metro feature
df.insert(6, 'time_to_get_metro', 0)
df.insert(7, 'by_car', 0)
df.insert(8, 'by_walk', 0)

In [12]:
#Check uniqueness of transport type and create lists for future distribution
splited_list_of_metro = [metro.split(' (') for metro in df['metro']]
lst_type_of_transport_metro = []
lst_time_to_get_metro = []
lst_name_of_metro = []
for i in splited_list_of_metro:
    lst_type_of_transport_metro.append(i[1].split()[-1].rstrip(')'))
    lst_time_to_get_metro.append(i[1].split()[0])
    lst_name_of_metro.append((i[0].split())[1])
    
unique_lst = pd.Series(lst_type_of_transport_metro).unique()
unique_lst

array(['пешком', 'машине'], dtype=object)

In [13]:
#Filling in 2 new columns by 1 if Metro has specific features related to columns
#And adding amount of min to get metro in 1 column
for i in range(len(splited_list_of_metro)):
    df.loc[i, 'metro'] = lst_name_of_metro[i]
    df.loc[i, 'time_to_get_metro'] = lst_time_to_get_metro[i]
    
    if lst_type_of_transport_metro[i] == 'пешком':
        df.loc[i, 'by_walk'] = 1
    else:
        df.loc[i, 'by_car'] = 1

/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_85011/3928166692.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'time_to_get_metro'] = lst_time_to_get_metro[i]


In [14]:
#Creating new columns for dataframe
df.insert(6, 'vao', 0)
df.insert(7, 'zao', 0)
df.insert(8, 'nao', 0)
df.insert(9, 'sao', 0)
df.insert(10, 'svao', 0)
df.insert(11, 'szao', 0)
df.insert(12, 'cao', 0)
df.insert(13, 'uvao', 0)
df.insert(14, 'uao', 0)
df.insert(15, 'uzao', 0)
df.insert(16, 'rayoni', 0)

In [15]:
#Creating lists of disctricts with related to them metro stations 
VAO = [
    'Авиамоторная',
    'Андроновка',
    'Белокаменная',
    'Бульвар Рокоссовского',
    'Измайлово',
    'Измайловская',
    'Локомотив',
    'Лухмановская',
    'Новогиреево',
    'Новокосино',
    'Партизанская',
    'Первомайская',
    'Перово',
    'Преображенская площадь',
    'Семеновская',
    'Соколиная Гора',
    'Сокольники',
    'Улица Дмитриевского',
    'Черкизовская',
    'Шоссе Энтузиастов',
    'Щелковская',
    'Электрозаводская',
    'Стромынка',
]

ZAO = [
    'Багратионовская',
    'Боровское шоссе',
    'Киевская',
    'Кропоткинская',
    'Крылатское',
    'Кунцевская',
    'Кутузовская',
    'Ломоносовский проспект',
    'Минская',
    'Мичуринский проспект',
    'Молодежная',
    'Новопеределкино',
    'Озёрная',
    'Озерная',
    'Парк Победы',
    'Пионерская',
    'Проспект Вернадского',
    'Рабочий Посёлок',
    'Рабочий поселок',
    'Рабочий посёлок',
    'Рабочий Поселок',
    'Раменки',
    'Сетунь',
    'Славянский бульвар',
    'Славянский бульвар',
    'Солнцево',
    'Студенческая',
    'Тропарёво',
    'Тропарево',
    'Филевский парк',
    'Фили',
    'Фили',
    'Юго-Западная',
    'Аминьевское шоссе',
    'Давыдково',
    'Дорогомиловская',
    'Кунцевская',
    'Проспект Вернадского',
    'Улица Новаторов',
]

NAO = [
    'Говорово',
    'Коммунарка',
    'Ольховая',
    'Остафьево',
    'Прокшино',
    'Рассказовка',
    'Румянцево',
    'Саларьево',
    'Филатов Луг',
    'Щербинка',
]

SAO = [
    'Аэропорт',
    'Балтийская',
    'Беговая',
    'Беломорская',
    'Бескудниково',
    'Верхние Лихоборы',
    'Водный стадион',
    'Войковская',
    'Гражданская',
    'Дегунино',
    'Динамо',
    'Дмитровская',
    'Зорге',
    'Коптево',
    'Красный Балтиец',
    'Лихоборы',
    'Марк',
    'Окружная',
    'Панфиловская',
    'Петровский парк',
    'Петровский Парк',
    'Петровско-Разумовская',
    'Покровское-Стрешнево',
    'Полежаевская',
    'Речной вокзал',
    'Савеловская',
    'Сокол',
    'Стрешнево',
    'Тимирязевская',
    'Ховрино',
    'Хорошёво',
    'Хорошево',
    'Хорошёвская',
    'Хорошевская',
    'ЦСКА',
    'Беломорская улица',
    'Петровский парк',
    'Петровско-Разумовская',
]

SVAO = [
    'Алексеевская',
    'Алтуфьево',
    'Бабушкинская',
    'Бибирево',
    'Ботанический сад',
    'Бутырская',
    'ВДНХ',
    'Владыкино',
    'Выставочный центр',
    'Лианозово',
    'Марьина роща',
    'Медведково',
    'Окружная',
    'Отрадное',
    'Ростокино',
    'Савеловская',
    'Свиблово',
    'Телецентр',
    'Улица Академика Королёва',
    'Улица академика королева',
    'Улица Академика королева',
    'Улица академика Королева',
    'Улица академика королёва',
    'Улица Академика королёва',
    'Улица академика Королёва',
    'Улица Милашенкова',
    'Улица Сергея Эйзенштейна',
    'Фонвизинская',
    'Бутырская',
    'Фонвизинская',
    'Шереметьевская',
]

SZAO = [
    'Волоколамская',
    'Митино',
    'Мнёвники',
    'Народное Ополчение',
    'Народное ополчение',
    'Октябрьское поле',
    'Пенягино',
    'Планерная',
    'Пятницкое шоссе',
    'Спартак',
    'Строгино',
    'Сходненская',
    'Трикотажная',
    'Тушинская',
    'Щукинская',
    'Нижние Мневники',
    'Терехово',
    'Улица Народного ополчения',
]

CAO = [
    'Александровский сад',
    'Арбатская',
    'Баррикадная',
    'Бауманская',
    'Беговая',
    'Белорусская',
    'Библиотека имени Ленина',
    'Боровицкая',
    'Воробьевы горы',
    'Выставочная',
    'Деловой центр',
    'Добрынинская',
    'Достоевская',
    'Каланчёвская',
    'Китай-город',
    'Комсомольская',
    'Краснопресненская',
    'Красносельская',
    'Красные ворота',
    'Крестьянская застава',
    'Кузнецкий мост',
    'Курская',
    'Лубянка',
    'Лужники',
    'Марксистская',
    'Маяковская',
    'Международная',
    'Менделеевская',
    'Новокузнецкая',
    'Новослободская',
    'Октябрьская',
    'Охотный ряд',
    'Павелецкая',
    'Парк Культуры',
    'Площадь Ильича',
    'Площадь Революции',
    'Полянка',
    'Пролетарская',
    'Проспект Мира',
    'Пушкинская',
    'Рижская',
    'Римская',
    'Серпуховская',
    'Смоленская',
    'Смоленская',
    'Спортивная',
    'Сретенский бульвар',
    'Сухаревская',
    'Таганская',
    'Тверская',
    'Театральная',
    'Тестовская',
    'Третьяковская',
    'Трубная',
    'Тургеневская',
    'Улицаа 1905 года',
    'Фрунзенская',
    'Цветной бульвар',
    'Чеховская',
    'Чистые пруды',
    'Чкаловская',
    'Шелепиха',
    'Электрозаводская',
    'Волхонка',
    'Плющиха',
    'Ржевская',
    'Рубцовская',
]

UVAO = [
    'Авиамоторная',
    'Братиславская',
    'Волгоградский проспект',
    'Волжская',
    'Выхино',
    'Депо',
    'Дубровка',
    'Жулебино',
    'Калитники',
    'Кожуховская',
    'Косино',
    'Котельники',
    'Кубанская',
    'Кузьминки',
    'Курьяново',
    'Лермонтовский проспект',
    'Лефортово',
    'Люблино',
    'Марьино',
    'Москва-Товарная',
    'Некрасовка',
    'Нижегородская',
    'Новохохловская',
    'Новохохловская',
    'Окская',
    'Перерва',
    'Печатники',
    'Рязанский проспект',
    'Стахановская',
    'Текстильщики',
    'Текстильщики',
    'Угрешская',
    'Юго-Восточная',
    'Печатники',
    'Текстильщики',
]

UZAO = [
    'Академическая',
    'Беляево',
    'Битцевский парк',
    'Бульвар адмирала Ушакова',
    'Бульвар Адмирала ушакова',
    'Бульвар адмирала ушакова',
    'Бульвар Дмитрия Донского',
    'Бунинская Аллея',
    'Бунинская аллея',
    'Бутово',
    'Калужская',
    'Каховская',
    'Коньково',
    'Нахимовский Проспект',
    'Нахимовский проспект',
    'Новоясеневская',
    'Новые Черёмушки',
    'Новые Черемушки',
    'Новые черёмушки',
    'Площадь Гагарина',
    'Профсоюзная',
    'Севастопольская',
    'Зюзино',
    'Теплый стан',
    'Теплый Стан',
    'Улица Горчакова',
    'Улица Скобелевская',
    'Улица Старокачаловская',
    'Университет',
    'Ясенево',
    'Воронцовская',
    'Каховская',
    'Севастопольский проспект',
]

UAO = [
    'Автозаводская',
    'Автозаводская',
    'Алма-Атинская',
    'Аннино',
    'Борисово',
    'Варшавская',
    'Верхние Котлы',
    'Домодедовская',
    'ЗИЛ',
    'Зябликово',
    'Кантемировская',
    'Каширская',
    'Коломенская',
    'Красногвардейская',
    'Красный Строитель',
    'Крымская',
    'Ленинский проспект',
    'Лесопарковая',
    'Москворечье',
    'Нагатинская',
    'Нагорная',
    'Орехово',
    'Покровское',
    'Пражская',
    'Технопарк',
    'Тульская',
    'Улица академика Янгеля',
    'Царицыно',
    'Царицыно',
    'Чертановская',
    'Шаболовская',
    'Шипиловская',
    'Южная',
    'Каширская',
    'Кленовый бульвар',
]

RAYONI = [
    'Аникеевка',
    'Баковка',
    'Битца',
    'Водники',
    'Долгопрудная',
    'Красногорская',
    'Лобня',
    'Мякинино',
    'мякинино',
    'Нахабино',
    'Немчиновка',
    'Новодачная',
    'Одинцово',
    'Опалиха',
    'Павшино',
    'Подольск',
    'Силикатная',
    'Сколково',
    'Хлебниково',
    'Шереметьевская',
]

In [16]:
#Creating dictionary to use it in function to distribute data into columns
region_map = {
    'vao': VAO,
    'zao': ZAO,
    'nao': NAO,
    'sao': SAO,
    'svao': SVAO,
    'szao': SZAO,
    'cao': CAO,
    'uvao': UVAO,
    'uzao': UZAO,
    'uao': UAO,
    'rayoni': RAYONI,
}

In [17]:
#Function which distributes data to columns respectively 
def update_regions(df):
    for region, metro_list in region_map.items():
        df[region] = df['metro'].apply(lambda x: 1 if x in metro_list else 0)
    return df

In [18]:
#Updating dataframe and deleting metro and address columns, because now we are using districts
df = update_regions(df)
df = df.drop('metro', axis=1)
df = df.drop('address', axis=1)

In [19]:
#Replacing all type of areas in area_m2 column by total area
df['area_m2'] = df['area_m2'].apply(lambda x: x.split('/')[0])

In [20]:
#Creating new columns for apt type
df.insert(21, 'monolithic_apt', 0)
df.insert(22, 'monolithic_brick_apt', 0)
df.insert(23, 'panel_apt', 0)
df.insert(24, 'stalin_apt', 0)
df.insert(25, 'brick_apt', 0)
df.insert(26, 'old_stock_apt', 0)
df.insert(27, 'block_apt', 0)
df.insert(28, 'wooden_apt', 0)
df.insert(29, 'aerated_concrete_block_apt', 0)
df.insert(30, 'shield_apt', 0)

In [21]:
#Distributing data to the new columns
splited_list_of_apt_desc = [i.split(', ') for i in df['apt_short_desc']]
lst_of_type_of_apt = []
lst_of_floor_of_apt = []

for i in range(len(splited_list_of_apt_desc)):
    lst_of_floor_of_apt.append(splited_list_of_apt_desc[i][0])

    if len(splited_list_of_apt_desc[i]) > 1:
        lst_of_type_of_apt.append(splited_list_of_apt_desc[i][1])

In [22]:
#continue of previous step
for i in range(len(lst_of_type_of_apt)):
    if lst_of_type_of_apt[i] == 'Монолитный':
        df.loc[i, 'monolithic_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Монолитно-кирпичный':
        df.loc[i, 'monolithic_brick_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Панельный':
        df.loc[i, 'panel_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Сталинский':
        df.loc[i, 'stalin_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Кирпичный':
        df.loc[i, 'brick_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'старый фонд':
        df.loc[i, 'old_stock_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Блочный':
        df.loc[i, 'block_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Деревянный':
        df.loc[i, 'wooden_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Газобетонный блок':
        df.loc[i, 'aerated_concrete_block_apt'] = 1
            
    elif lst_of_type_of_apt[i] == 'Щитовой':
        df.loc[i, 'shield_apt'] = 1

In [23]:
#Taking out the floor of the apt and placing into list
sorted_lst_of_floor_of_apt = [i.split('/')[0] for i in lst_of_floor_of_apt]

In [24]:
df.insert(31, 'apt_floor', 0)

In [25]:
for i in range(len(sorted_lst_of_floor_of_apt)):
    df.loc[i, 'apt_floor'] = sorted_lst_of_floor_of_apt[i]

/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_85011/717303396.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'apt_floor'] = sorted_lst_of_floor_of_apt[i]


In [26]:
#Rid of apt_short_desc column
df = df.drop('apt_short_desc', axis=1)

In [27]:
#Looking for string None and replacing it by mode of the column
columns_to_process = ['number_of_rooms', 'time_to_get_metro', 'area_m2', 'apt_floor']

def replace_none_with_mode(df, columns):
    for column in columns:
        # Convert column to numeric, coerce errors to NaN
        df[column] = pd.to_numeric(df[column], errors='coerce')
        # Calculate the mode
        mode_value = df[column].mode()[0]
        # Replace 'None' with the mode
        df[column].fillna(mode_value, inplace=True)
    return df

df = replace_none_with_mode(df, columns_to_process)

/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_85011/2663852186.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mode_value, inplace=True)
/var/folders/db/69n9mbnx1hn26gcpf46v9d8h0000gn/T/ipykernel_85011/2663852186.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

In [57]:
#Converting columns to float
columns_to_convert = ['number_of_rooms', 'isolated_apt', 'adjacent_apt', 'iso_adj_apt',
                      'vao', 'zao', 'nao', 'sao', 'svao', 'szao', 'cao', 'uvao', 'uao',
                      'uzao', 'rayoni', 'time_to_get_metro', 'by_car', 'by_walk',
                      'area_m2', 'monolithic_apt', 'monolithic_brick_apt', 'panel_apt',
                      'stalin_apt', 'brick_apt', 'old_stock_apt', 'block_apt', 'wooden_apt',
                      'aerated_concrete_block_apt', 'shield_apt', 'apt_floor']

df[columns_to_convert] = df[columns_to_convert].astype(float)

In [61]:
df.insert(31, 'time_to_subway', 0)

In [62]:
df.loc[:, 'post_id':'time_to_subway']

,post_id,number_of_rooms,isolated_apt,adjacent_apt,iso_adj_apt,vao,zao,nao,sao,svao,szao,cao,uvao,uao,uzao,rayoni,time_to_get_metro,by_car,by_walk,area_m2,monolithic_apt,monolithic_brick_apt,panel_apt,stalin_apt,brick_apt,old_stock_apt,block_apt,wooden_apt,aerated_concrete_block_apt,shield_apt,apt_floor,time_to_subway
0,271271157,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,0.0,1.0,200.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0
1,271634126,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,198.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0
2,271173086,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,200.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0
3,272197456,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,170.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0
4,273614615,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0
23364,274654844,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,38.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0
23365,268679909,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,43.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0
23366,274807525,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,1.0,52.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0


In [63]:
#Managing time that takes to get to the metro
df['time_to_subway'] = df.apply(
    lambda row: row['time_to_get_metro'] if row['by_car'] == 0 else row['time_to_get_metro'] * 2,
    axis=1
)

In [66]:
#Delete unnecessary columns
df = df.drop('time_to_get_metro', axis=1)
df = df.drop('by_car', axis=1)
df = df.drop('by_walk', axis=1)

In [68]:
df.loc[:, 'post_id':'time_to_subway']

,post_id,number_of_rooms,isolated_apt,adjacent_apt,iso_adj_apt,vao,zao,nao,sao,svao,szao,cao,uvao,uao,uzao,rayoni,area_m2,monolithic_apt,monolithic_brick_apt,panel_apt,stalin_apt,brick_apt,old_stock_apt,block_apt,wooden_apt,aerated_concrete_block_apt,shield_apt,apt_floor,time_to_subway
0,271271157,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,200.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,9.0
1,271634126,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,198.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,8.0
2,271173086,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,7.0
3,272197456,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,170.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0
4,273614615,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,58.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,8.0
23364,274654844,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,7.0
23365,268679909,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6.0
23366,274807525,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,11.0


Bul's part

In [32]:
temp1=df['Цена']
rent_price_curr = temp1.apply(lambda x:x.split('/')[0].split())
rent_price = rent_price_curr.apply(lambda x:x[0])
# Цена аренды

In [33]:
currency = rent_price_curr.apply(lambda x:x[1]) # Валюта аренды

In [34]:
s3=temp1.apply(lambda x:x.split('Срок аренды - '))
stay_time=s3.apply(lambda x:x[1].split(',')[0]) # Виды сроков

In [35]:
deposite = temp1.apply(lambda x:x.split('Залог - '))
test1 = deposite.apply(lambda x:len(x))
bool_series1 = test1==2

deposite[bool_series1] = deposite[bool_series1].apply(lambda x:x[1].split(',')[0].split(' '))
deposite[~bool_series1] = '00'

deposite_value = deposite.apply(lambda x:x[0]) # Все залоги, если нету = то 0 строкой
deposite_currency = deposite.apply(lambda x:x[1]) # Все валюты, если нету = то 0 строкой

In [36]:
pre_pay = temp1.apply(lambda x:x.split('Предоплата ')[-1])

pre_pay_months = pre_pay.apply(lambda x:x.split(' ')[0]) 

pre_pay_bool = pre_pay.apply(lambda x:x.split(' ')[-1])
pre_pay_bool[(pre_pay_bool=='Длительный')|(pre_pay_bool=='месяцев')]='нету'
# Предоплата либо "мес", либо нету - заменил на "нету".

pre_pay_months = pre_pay_months.astype(float)
pre_pay_months[pre_pay_months>12]=0
# Количество месяцев

In [37]:
temp2=df['Ремонт']
temp2.value_counts(dropna=False)

In [38]:
temp4=df['Балкон']
temp4.value_counts(dropna=False)

In [39]:
df = df.drop(['Цена'], axis=1) # Убераем цену

In [40]:
df.insert(12,'rent_price',rent_price)
df.insert(13,'rent_currency',currency)
df.insert(14,'stay_time',stay_time)
df.insert(15,'deposite',deposite_value)
df.insert(16,'deposite_currency',deposite_currency)
df.insert(17,'is_prepay',pre_pay_bool)
df.insert(18,'prepay_months',pre_pay_months)

In [41]:
df = df.drop(['Описание'], axis=1) # Убераем описание

In [42]:
df['rent_price']=df['rent_price'].astype(float)
df['deposite']=df['deposite'].astype(float)
# Переводит числовые во float

In [43]:
df = df.drop(['is_prepay'], axis=1) # Убераем логический признак
df['prepay_months'] = df['prepay_months'].astype(int) # Теперь если 0 - то нету Предоплаты, или количество месяцев предоплаты

In [44]:
df = df.rename(columns={'Ремонт': 'renovation'}) # Переименовываем колонку

In [45]:
df['renovation']=df['renovation'].fillna('Нету') # Пропуски = "Нету"

In [46]:
df = df.rename(columns={'Балкон': 'balcony'}) # Переименовываем колонку

In [47]:
df['balcony']=df['balcony'].fillna('Балкон (0), Лоджия (0)') # Пропуски = 'Балкон (0), Лоджия (0)'

Nastya's part

In [48]:
# df['pets_okay'] = df['Можно с детьми/животными'].apply(lambda x: 1 if isinstance(x, str) and 'Можно с животными' in x else (0 if isinstance(x, str) else np.nan))
# mode_pets_okay = df['pets_okay'].mode()[0]
# df['pets_okay'].fillna(mode_pets_okay, inplace=True)
# print(df['pets_okay'])

In [49]:
# percent_counts = df['pets_okay'].value_counts(normalize=True) * 100
# # Print the percentage counts
# print("Percentage of each value in 'pets_okay':")
# print(percent_counts)

In [50]:
# df['kids_okay'] = df['Можно с детьми/животными'].apply(lambda x: 1 if isinstance(x, str) and 'Можно с детьми' in x else (0 if isinstance(x, str) else np.nan))
# mode_kids_okay = df['kids_okay'].mode()[0] # 🚧
# df['kids_okay'].fillna(mode_kids_okay, inplace=True)
# print(df['kids_okay'])

In [51]:
# percent_counts = df['kids_okay'].value_counts(normalize=True) * 100

# # Print the percentage counts
# print("Percentage of each value in 'kids_okay':")
# print(percent_counts)

In [52]:
# def create_column(df, new_col_name, phrase):
#     df[new_col_name] = df['Дополнительно'].apply(
#         lambda x: 1 if isinstance(x, str) and phrase in x else (0 if isinstance(x, str) else np.nan)
#     )
#     # Fill NaN values with the mode
#     mode_value = df[new_col_name].mode()[0]
#     df[new_col_name].fillna(mode_value, inplace=True)
#     return df

# # Create new columns
# df = create_column(df, 'furniture_in_the_room', 'Мебель в комнатах')
# df = create_column(df, 'air_conditioner', 'Кондиционер')
# df = create_column(df, 'dishwashing', 'Посудомоечная машина')
# df = create_column(df, 'fridge', 'Холодильник')
# df = create_column(df, 'internet', 'Интернет')

In [53]:
# # Checking descriptive stats for each main feature
# def calculate_percentage(df, column_name):
#     percent_counts = df[column_name].value_counts(normalize=True) * 100
#     print(f"Percentage of each value in '{column_name}':")
#     print(percent_counts)

# calculate_percentage(df, 'furniture_in_the_room')
# calculate_percentage(df, 'air_conditioner')
# calculate_percentage(df, 'dishwashing')
# # Doing reality check - these columns should largely have 1 
# calculate_percentage(df, 'fridge')
# calculate_percentage(df, 'internet')


# print(df[['furniture_in_the_room', 'air_conditioner', 'dishwashing', 'fridge', 'internet']])

In [54]:
# df['only_street_view'] = df['Окна'].apply(
#     lambda x: 1 if isinstance(x, str) and 'На улицу' in x else (0 if isinstance(x, str) else np.nan)
# )
# mode_only_street_view = df['only_street_view'].mode()[0]
# df['only_street_view'].fillna(mode_only_street_view, inplace=True)
# percent_counts = df['only_street_view'].value_counts(normalize=True) * 100

# print("Percentage of each value in 'only_street_view':")
# print(percent_counts)

In [55]:
# #Getting rid of columns 
# df = df.drop(['Окна'], axis=1)
# df = df.drop(['Можно с детьми/животными'], axis=1)
# df = df.drop(['Дополнительно'], axis=1)
# df = df.drop(['Название ЖК'], axis=1)
# df = df.drop(['Серия дома'], axis=1)
# df = df.drop(['Ссылка на объявление'], axis=1)


In [56]:
# df.to_csv('../data/data.csv')